In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {char: i for i, char in enumerate((world_set))} #인덱스와 해당 인덱스의 값(문자)를 가져와서 dict형태로 설정

In [4]:
print(vocab)

{'w': 0, "'": 1, 'l': 2, 'y': 3, 'c': 4, 's': 5, 'a': 6, 'h': 7, 'B': 8, 'f': 9, 'p': 10, ',': 11, 'e': 12, 'i': 13, 'u': 14, 'g': 15, 'r': 16, '.': 17, 'n': 18, 'o': 19, ' ': 20, 'm': 21, 'b': 22, 't': 23, 'd': 24, 'k': 25}


In [5]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [6]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

# hidden size  : hidden layer의 각 노드가 처리할 수 있는 정보의 차원 수

- hidden_size가 크면 **더 많은 정보**를 내부 상태에 저장할 수 있어서 더 복잡한 패턴을 학습할 수 있지만, **과적합 발생** 가능성 있음.
- hidden_size가 작으면 모델의 **계산 효율성**은 높아지지만, 너무 적은 정보만을 저장할 수 있어서 **학습 능력이 제한**될 수 있음.


--> 어휘 크기가 상대적으로 작기 때문에 vocab_size와 같게 설정해서 모델이 충분한 정보를 저장할 수 있도록 함.



In [7]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

In [8]:
## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[8, 16, 13, 4, 25, 20, 0, 6, 2, 2, 5, 20, 6, 16, 12]
[16, 13, 4, 25, 20, 0, 6, 2, 2, 5, 20, 6, 16, 12, 20]


In [10]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
N = vocab_size
x_one_hot= [np.eye(N)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-10-15539bf880af>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.FloatTensor(x_one_hot)


In [11]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([183, 15, 26])
레이블의 크기 : torch.Size([183, 15])


In [12]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [13]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([16, 13,  4, 25, 20,  0,  6,  2,  2,  5, 20,  6, 16, 12, 20])


In [14]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=layers,
                            batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    x, hidden  = self.rnn(x)
    x = self.fc(x)
    return x

In [15]:
net = Net(vocab_size, hidden_size, 2)

In [16]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [17]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([183, 15, 26])


In [18]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

#16. 예측결과 확인
results = outputs.argmax(dim=2) # 텐서의 마지막 차원(클래스 차원)을 따라 최댓값의 인덱스
predict_str = ""
for j, result in enumerate(results): #각 문자에 대한 예측 결과와 인덱스 추출
    if j == 0: # 처음에는 예측 결과를 전부 가져오기
        predict_str += ''.join([world_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
        predict_str += world_set[result[-1]]

print(predict_str)

rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thinksl


In [19]:
results.shape

torch.Size([183, 15])

In [20]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thinksl"

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

#### 연설문 데이터

안녕하십니까.
부총리 겸 교육부장관 이주호입니다.

오늘 오전, 대통령께서 주재하는 ‘국민과 함께하는 민생토론회’에 참석하여, 학부모, 교원 등 다양한 국민들의 의견을 들었습니다.

대통령께서는 이 자리에서 이제는 ‘부모 돌봄에서 국가 돌봄으로 나아가야 한다.’라고 하시면서 늘봄학교의 중요성을 강조하셨습니다.

또한, 국가 돌봄이 정착하려면 ‘무엇보다 공교육의 중심인 학교의 역할이 확대돼야 한다.’라고도 하셨습니다.

출처 : https://www.korea.kr/briefing/speechView.do?newsId=132036217&pageIndex=1&srchType=&period=&startDate=2023-02-12&endDate=2024-02-12&srchKeyword=

### sequence_length = 20 + layer 3 + 에폭100


### 1. 텍스트 전처리

In [21]:
#1. 데이터 정의 및 토큰화

# 연설문 데이터
data = "안녕하십니까. 부총리 겸 교육부장관 이주호입니다. 오늘 오전, 대통령께서 주재하는 ‘국민과 함께하는 민생토론회’에 참석하여, 학부모, 교원 등 다양한 국민들의 의견을 들었습니다. 대통령께서는 이 자리에서 이제는 ‘부모 돌봄에서 국가 돌봄으로 나아가야 한다.’라고 하시면서 늘봄학교의 중요성을 강조하셨습니다. 또한, 국가 돌봄이 정착하려면 ‘무엇보다 공교육의 중심인 학교의 역할이 확대돼야 한다.’라고도 하셨습니다."


In [22]:
# 간단한 토큰화 : 공백으로 분리
tokens = data.split()

# 고유 토큰 집합 생성 및 정수 인코딩
vocab = {tkn: i for i, tkn in enumerate(set(tokens))}
vocab_size = len(vocab)

print(vocab)

{'또한,': 0, '오전,': 1, '함께하는': 2, '이': 3, '‘무엇보다': 4, '중심인': 5, '국가': 6, '학부모,': 7, '확대돼야': 8, '돌봄이': 9, '정착하려면': 10, '늘봄학교의': 11, '안녕하십니까.': 12, '참석하여,': 13, '학교의': 14, '의견을': 15, '민생토론회’에': 16, '국민들의': 17, '등': 18, '하시면서': 19, '대통령께서': 20, '교원': 21, '주재하는': 22, '중요성을': 23, '들었습니다.': 24, '역할이': 25, '자리에서': 26, '부총리': 27, '다양한': 28, '이주호입니다.': 29, '교육부장관': 30, '강조하셨습니다.': 31, '‘국민과': 32, '한다.’라고도': 33, '오늘': 34, '돌봄으로': 35, '하셨습니다.': 36, '이제는': 37, '나아가야': 38, '공교육의': 39, '겸': 40, '한다.’라고': 41, '대통령께서는': 42, '돌봄에서': 43, '‘부모': 44}


In [23]:
# 데이터를 시퀀스로 변환 ()
sequences = [vocab[tkn] for tkn in tokens]

In [24]:
#문자 집합 크기 확인
vocab_size = len(vocab) #26개의 고유한 문자가 있음
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 45


In [25]:
# 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size
sequence_length = 20
learning_rate = 0.01

In [26]:
# eqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = [] #입력 시퀀스
y_data = [] #각 입력 시퀀스에 대응하는 타깃 문자(입력 시퀀스 바로 다음에 오는 문자)

for i in range(len(sequences) - sequence_length):
    x_data.append(sequences[i:i+sequence_length])
    y_data.append(sequences[i+1:i+sequence_length+1])

##입력 시퀀스에 대해 원핫인코딩 수행
N = vocab_size  #191
x_one_hot= [np.eye(N)[x] for x in x_data]

# 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [27]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[12, 27, 40, 30, 29, 34, 1, 20, 22, 32, 2, 16, 13, 7, 21, 18, 28, 17, 15, 24]
[27, 40, 30, 29, 34, 1, 20, 22, 32, 2, 16, 13, 7, 21, 18, 28, 17, 15, 24, 42]


In [28]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([26, 20, 45])
레이블의 크기 : torch.Size([26, 20])


In [29]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0]) #4개 각각 문자의 바로 다음 문자에 대한 인덱스 (정답)

tensor([27, 40, 30, 29, 34,  1, 20, 22, 32,  2, 16, 13,  7, 21, 18, 28, 17, 15,
        24, 42])


### 2. 모델 정의

In [30]:
##11. RNN 모델 구현

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=layers,
                            batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    x, hidden  = self.rnn(x)
    x = self.fc(x)
    return x

In [31]:
net = Net(vocab_size, hidden_size, 3)

In [32]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [33]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([26, 20, 45])


### 3. 모델 학습

In [34]:
index_to_token = {index: token for token, index in vocab.items()}

##15. Training 시작

for i in range(100): #에폭 100
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward() #역전파를 수행하여 각 파라미터의 그라디언트를 계산
    optimizer.step() #파라미터 업데이트

    #16. 예측결과 확인
results = outputs.argmax(dim=2) # 텐서의 마지막 차원(클래스 차원)을 따라 최댓값의 인덱스 -> 183 * 15개의 인덱스
predict_str = ""
for j, result in enumerate(results):
  if j == 0:  # 첫 번째 결과에 대해서는 전체 문자열을 생성
    predict_str += ''.join([index_to_token[t.item()] for t in result])
else:  # 그 다음부터는 마지막 글자만 추가
    predict_str += index_to_token[result[-1].item()]


### 4. 예측 결과 확인


In [35]:
predict_str

'부총리겸교육부장관이주호입니다.오늘오전,대통령께서주재하는‘국민과함께하는민생토론회’에참석하여,학부모,교원등다양한국민들의의견을들었습니다.대통령께서는하셨습니다.'